<a href="https://colab.research.google.com/github/rautshree/AI-class/blob/master/Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy
from keras.datasets import imdb
from keras.models import Sequential, Model
from keras.layers import Dense
from keras.layers import LSTM, Convolution1D, Flatten, Dropout
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence

Using TensorFlow backend.


For this analysis we’ll be using a dataset of 50,000 movie reviews taken from IMDb. The data was compiled by Andrew Maas and can be found here: [IMDb Reviews.](http://ai.stanford.edu/~amaas/data/sentiment/)

The data is split evenly with 25k reviews intended for training and 25k for testing your classifier. Moreover, each set has 12.5k positive and 12.5k negative reviews.

IMDb lets users rate movies on a scale from 1 to 10. To label these reviews the curator of the data labeled anything with ≤ 4 stars as negative and anything with ≥ 7 stars as positive. Reviews with 5 or 6 stars were left out.

Ofcourse, we can download it and do the pre-processing ourselves, well well well Keras already provides a wrapper over pre-processed dataset.

In [2]:
top_words = 10000
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=top_words)

17465344/17464789 [==============================] - 0s 0us/step


In [3]:
# These words are encoded into numbers by the index of their place in the vocabulary.
print (X_train[0], y_train[0])

[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32] 1


In [4]:
# Let us decode a sentence to see what it holds
d = imdb.get_word_index()
inv_map = {v: k for k, v in d.items()}
for i in X_train[0][1:]:
  if (i>=3):
    print (inv_map[i-3])

1646592/1641221 [==============================] - 0s 0us/step
this
film
was
just
brilliant
casting
location
scenery
story
direction
everyone's
really
suited
the
part
they
played
and
you
could
just
imagine
being
there
robert
is
an
amazing
actor
and
now
the
same
being
director
father
came
from
the
same
scottish
island
as
myself
so
i
loved
the
fact
there
was
a
real
connection
with
this
film
the
witty
remarks
throughout
the
film
were
great
it
was
just
brilliant
so
much
that
i
bought
the
film
as
soon
as
it
was
released
for
and
would
recommend
it
to
everyone
to
watch
and
the
fly
fishing
was
amazing
really
cried
at
the
end
it
was
so
sad
and
you
know
what
they
say
if
you
cry
at
a
film
it
must
have
been
good
and
this
definitely
was
also
to
the
two
little
boy's
that
played
the
of
norman
and
paul
they
were
just
brilliant
children
are
often
left
out
of
the
list
i
think
because
the
stars
that
play
them
all
grown
up
are
such
a
big
profile
for
the
whole
film
but
these
children
are
amazing
and
should

In [0]:
max_review_length = 80 # ? You need to change this, maybe 100, maybe 1000, how do you find it? Maybe the max length of reviews or maybe something for faster training! Do you always need to compromise in life?
X_train = sequence.pad_sequences(X_train, maxlen=max_review_length)
X_test = sequence.pad_sequences(X_test, maxlen=max_review_length)#?? # What if we don't pad the test data? 

In [6]:
print (X_train[0], len(X_train[0]))


[  15  256    4    2    7 3766    5  723   36   71   43  530  476   26
  400  317   46    7    4    2 1029   13  104   88    4  381   15  297
   98   32 2071   56   26  141    6  194 7486   18    4  226   22   21
  134  476   26  480    5  144   30 5535   18   51   36   28  224   92
   25  104    4  226   65   16   38 1334   88   12   16  283    5   16
 4472  113  103   32   15   16 5345   19  178   32] 80


![Conv1D](https://cdn-images-1.medium.com/max/1600/1*aBN2Ir7y2E-t2AbekOtEIw.png)


In [7]:
embedding_vector_length = 300
model = Sequential()
model.add(Embedding(top_words, embedding_vector_length, input_length=max_review_length))
# Some bad architecture choice here, Could you please try to find what's wrong?
model.add(Convolution1D(64, 3, padding='same')) # Why padding is same? https://keras.io/layers/convolutional/
model.add(Convolution1D(32, 3, padding='same'))
model.add(Convolution1D(16, 3, padding='same'))
model.add(Flatten())
model.add(Dropout(0.2)) # Should you increase this for more generalization?
model.add(Dense(180,activation='sigmoid'))
model.add(Dropout(0.2))
model.add(Dense(1,activation='sigmoid')) # Can we use 2 here?
print (model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 80, 300)           3000000   
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 80, 64)            57664     
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 80, 32)            6176      
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 80, 16)            1552      
_________________________________________________________________
flatten_1 (Flatten)          (None, 1280)              0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 1280)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 180)               230580    
__________

In [0]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [9]:
model.fit(X_train, y_train, nb_epoch=2, verbose = 1, batch_size = 64)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  """Entry point for launching an IPython kernel.


Epoch 1/2
25000/25000 [==============================] - 54s 2ms/step - loss: 0.4438 - acc: 0.7797
Epoch 2/2
25000/25000 [==============================] - 52s 2ms/step - loss: 0.2386 - acc: 0.9039


In [10]:
loss_acc = model.evaluate(X_test, y_test, verbose=1)
print("Test data: loss = %0.6f  accuracy = %0.2f%% " % \
  (loss_acc[0], loss_acc[1]*100))

25000/25000 [==============================] - 12s 491us/step
Test data: loss = 0.418050  accuracy = 82.32% 


In [11]:
# 5. save model
print("Saving model to disk \n")
mp = ".\\Models\\imdb_model.h5"
model.save(mp)

Saving model to disk 



In [26]:
print("New review: \'the school was good\'")
review = "the school was good"
words = review.split()
review = []
for word in words:
  if word not in d: 
    review.append(2)
  else:
    review.append(d[word]+3) 

New review: 'the school was good'


In [0]:
review = [0]*(max_review_length-len(review)) + review

In [28]:
# review = sequence.pad_sequences(review, maxlen=80)
import numpy as np
review = np.array(review)
print (review.shape)
prediction = model.predict(review.reshape(1,80))
print("Prediction (0 = negative, 1 = positive) = ", end="")
print("%0.4f" % prediction[0][0])

(80,)
Prediction (0 = negative, 1 = positive) = 0.3972


Now let's  try an LSTM based model, **could you change this to BiLSTM model?**

LSTMs and their bidirectional variants are popular because they have tried to learn how and when to forget and when not to using gates in their architecture. In previous RNN architectures, vanishing gradients was a big problem and caused those nets not to learn so much.

Using Bidirectional LSTMs, you feed the learning algorithm with the original data once from beginning to the end and once from end to beginning.

In [21]:
import keras as K
model = K.models.Sequential()
model.add(K.layers.embeddings.Embedding(input_dim=top_words,
  output_dim=embedding_vector_length, mask_zero=True))
model.add(K.layers.LSTM(units=100, dropout=0.2, recurrent_dropout=0.2))  # 100 memory
model.add(K.layers.Dense(units=1, activation='sigmoid'))
print (model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, None, 300)         3000000   
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               160400    
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 101       
Total params: 3,160,501
Trainable params: 3,160,501
Non-trainable params: 0
_________________________________________________________________
None


In [0]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [23]:
model.fit(X_train, y_train, nb_epoch=2, verbose = 1, batch_size = 64)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  """Entry point for launching an IPython kernel.


Epoch 1/2
25000/25000 [==============================] - 152s 6ms/step - loss: 0.4576 - acc: 0.7844
Epoch 2/2
25000/25000 [==============================] - 151s 6ms/step - loss: 0.3182 - acc: 0.8689


In [24]:
loss_acc = model.evaluate(X_test, y_test, verbose=1)
print("Test data: loss = %0.6f  accuracy = %0.2f%% " % \
  (loss_acc[0], loss_acc[1]*100))

25000/25000 [==============================] - 53s 2ms/step
Test data: loss = 0.376176  accuracy = 84.05% 
